In [ ]:
### 줄거리로 유사도 분석을 한다.
### 어떤 서비스를 제공하려고 하는냐? 클라이언트가 영화를 클릭하면 클릭한 영화와 유사한 
### 영화 10개를 추천해 주려고 한다. 이때 기준은 줄거리가 유사하면 추천한다.

## 개발순서 
# 1. 데이터를 가져온다.. 우리가 가져올 데이터는 tmdb_5000_movies.csv
# 2. 유사도를 분석할 데이터만 정제한다.. 우리가 사용할 컬럼은 overview 
# 3. overview로 document-term matrix를 만든다..
#     행은 각각의 문서, 옅은 각각의 단어  * 열을 보면은 문서에 단어가 카운트된 내용을 알 수 있다.
#     체크사항: 단어를 백터화한 내용, document term martrix 구분하여 이해 
# 4. 3에서 만든 document-term matrix로 유사도를 분석합니다.  코사인유사도 알고리즘을 사용 
# 5. 4에서 유사도를 분석한 결과로 유사도가 높은 10개의 영화를 선정
# 6. 5번에서 선정된 영화를 영화의 이름이 아닌 번호입니다. 그래서 이름과 번호를 매칭시키도록 한다.
# 7. 테스트: 4번 영화라고 가정을 하고 4번 영화와 유사도가 높은 영화 10개를 선정하여 출력한다.

In [2]:
# 1. 데이터를 가져온다.. 우리가 가져올 데이터는 tmdb_5000_movies.csv
# 사용할 라이브러리는 pandas > 2차원 백터이므로 dataframe 타입
import pandas as pd

df1 = pd.read_csv('./data/tmdb_5000_movies.csv')
df1.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [4]:
# 2. 유사도를 분석할 데이터만 정제한다.. 우리가 사용할 컬럼은 overview 
df2=df1[['original_title','overview']]
print(type(df2))
df2 

<class 'pandas.core.frame.DataFrame'>


,original_title,overview
0,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,Spectre,A cryptic message from Bond’s past sends him o...
3,The Dark Knight Rises,Following the death of District Attorney Harve...
4,John Carter,"John Carter is a war-weary, former military ca..."
...,...,...
4798,El Mariachi,El Mariachi just wants to play his guitar and ...
4799,Newlyweds,A newlywed couple's honeymoon is upended by th...
4800,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic..."
4801,Shanghai Calling,When ambitious New York attorney Sam is sent t...


In [ ]:
### 유사도 분석하기 위한 데이터와 - 유사도를 분석하면 2차원 행렬(데이터 프레임이아닙니다.) 즉, 인덱스와 컬럼명이 없다) 
### 유사도 분석을 하면 overview의 위치가 있는데 그 위치를 영화의 제목을 매칭하기 위한 데이터를 분리 
co_data = df2['overview']
co_data  ## 유사도를 분석하기 위한 데이터
mo_data = df2['original_title']
mo_data  ## 영화 인덱스와 매칭 되는 영화의 제목을 저장한 데이터

0                                         Avatar
1       Pirates of the Caribbean: At World's End
2                                        Spectre
3                          The Dark Knight Rises
4                                    John Carter
                          ...                   
4798                                 El Mariachi
4799                                   Newlyweds
4800                   Signed, Sealed, Delivered
4801                            Shanghai Calling
4802                           My Date with Drew
Name: original_title, Length: 4803, dtype: object

In [ ]:
## 3. overview로 document-term matrix를 만든다 
## 위에서 만든 co_data로 document-term matrix 만든다 이 것으로 나중에 유사도 분석을 하겠다 
## 만약 줄거리가 없는 경우(null)는 분석에 오류가 많이 발생한다 
## 해결하는 방법은 그 로우를 지우는 방법이 있는데.. 이렇게 되면 인덱스명과 번호가 불일치 해서 안됨 
## 해결하는 방법으로 null값을 빈값으로 채우는 것을 선택한다.

## 데이터 전처리시 널값은 필수라고 생각하면 됩니다.
co_data.isnull() # 널 값이 있는 지 확인해라 
co_data.isnull().values
co_data.isnull().values.any() #결과가 True 라면 null값이 있다는 이야기 입니다

array([False, False, False, ..., False, False, False], shape=(4803,))

In [9]:
# 널값이 존재한다.. 널 값이 있는 행을 지우면 안된다.. 그래서 우리는 빈칸으로 채우기로 함 
co_data = co_data.fillna('')

In [10]:
#널값이 없는지 다시 확인 
co_data.isnull().values.any() #결과가 True 라면 null값이 있다는 이야기 입니다

np.False_

In [13]:
### 매트릭스를 만들기 위해서는 라이브러리가 필요하다..
## 사용할 라이브러리는 사이킷런 : 인공지능 모델 개발에 많이 사용되는 라이브러리 입니다.
# pip install scikit learn
# document term matrix
from sklearn.feature_extraction.text import TfidfVectorizer
# 줄거리를 document term matrix로 백터화 시켜야 한다. 이때 사용할 알고리즘이 TfidfVectorizer로 선정함 
# 이외에도 다양한 알고리즘이 있지만. 현재는 이것을 사용합니다 
# 기능, 특정 단어가 문서에 나오는 빈도, 범용적인 단어의 중요를 낮춰서 백터화 시키는 특징 < 백터화의 정확도가 높다 

In [14]:
# 백터화를 시켜보겠다 
tf = TfidfVectorizer(stop_words='english') # 백터를 할 때 단어의 기준은 english이다.
tf_matrix = tf.fit_transform(co_data) # overview로 백터화 만들기 

In [ ]:
# 백터화 확인 
tf_matrix  ## 4803개의 문장이 있고, 전체 문장에서 분리한 단어가 20978개가 있다

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 125840 stored elements and shape (4803, 20978)>

In [ ]:
## 매트릭스를 확인 
a = tf_matrix.toarray()
a[0]  ## 0번째 문장의 백터화 정보를.. 컬럼 .. 20978개...

array([0., 0., 0., ..., 0., 0., 0.], shape=(20978,))

In [ ]:
## 백터화 단어와 카운트 확인 
tf.vocabulary_

{'22nd': 225,
 'century': 3232,
 'paraplegic': 13680,
 'marine': 11667,
 'dispatched': 5519,
 'moon': 12411,
 'pandora': 13644,
 'unique': 19671,
 'mission': 12269,
 'torn': 19054,
 'following': 7414,
 'orders': 13355,
 'protecting': 14754,
 'alien': 795,
 'civilization': 3545,
 'captain': 2999,
 'barbossa': 1733,
 'long': 11220,
 'believed': 1951,
 'dead': 4854,
 'come': 3843,
 'life': 11025,
 'headed': 8674,
 'edge': 6069,
 'earth': 6016,
 'turner': 19418,
 'elizabeth': 6188,
 'swann': 18376,
 'quite': 14969,
 'cryptic': 4588,
 'message': 12051,
 'bond': 2353,
 'past': 13749,
 'sends': 16695,
 'trail': 19141,
 'uncover': 19550,
 'sinister': 17127,
 'organization': 13364,
 'battles': 1824,
 'political': 14308,
 'forces': 7442,
 'secret': 16611,
 'service': 16753,
 'alive': 806,
 'peels': 13845,
 'layers': 10854,
 'deceit': 4909,
 'reveal': 15769,
 'terrible': 18739,
 'truth': 19371,
 'spectre': 17578,
 'death': 4878,
 'district': 5573,
 'attorney': 1439,
 'harvey': 8608,
 'dent': 5109

In [ ]:
### 여기까지는 백터화라는 개념을 이해하기 위한 코드이고 
## 다음 과정인 유사도를 분석하기 위해서는 우리는 document term matrix 가 필요하고 
## 이 매트릭스를 저장하는 변수는 tf_matrix입니다.  4803개의 문장가 약 2만의 컬럼이 있다..

In [19]:
##  tf_matrix 백터의 정보로 유사도를 분석해 보겠습니다..
## 알고리즘은 사이킷런에서 제공하는 코사인유사도 알고리즘 사용합니다 
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tf_matrix, tf_matrix)

In [20]:
## 코사인 유사도 분석한 결과를 확인 
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.02160533, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.01488159, 0.        ,
        0.        ],
       ...,
       [0.        , 0.02160533, 0.01488159, ..., 1.        , 0.01609091,
        0.00701914],
       [0.        , 0.        , 0.        , ..., 0.01609091, 1.        ,
        0.01171696],
       [0.        , 0.        , 0.        , ..., 0.00701914, 0.01171696,
        1.        ]], shape=(4803, 4803))

In [ ]:
cosine_sim[3] ## cosine_sim 배열의 4번째 인덱스의 값.. 즉, cosine_sim의 4번째 행의 값
              ## 4번째 행의 값이지.. 4번의 영화의 정보는 아니라는 것입니다.
              ## 그런데 위에서 mo_data를 만든 이유가 cosine_sim의 인덱스랑 매칭시켜서 영화의 정보를 알고 싶어서 만듬
print(cosine_sim[3])
mo_data[3]  # mo_data의 3번 인덱스명...
            # cosine_sim의 인덱스명과 일치해서 봐도 무방하다는 것이다.


## cosine_sim 배열에서 4번째 행은, mo_data의 4번째 영화의 유사도 분석 결과이다..              

[0.02499512 0.         0.         ... 0.03386366 0.04275232 0.02269198]


'The Dark Knight Rises'

In [ ]:
# 5. 4에서 유사도를 분석한 결과로 유사도가 높은 10개의 영화를 선정

aa = cosine_sim[3] # 3번째 영화가 아닌 3번째 인덱스의 유사도만 가져온 것..
aa_sort = sorted(aa, reverse=True)
aa_sort            ## 내림차순으로 정렬된 모든 정보가 출력이되고 
aa_sort[1:11]       ## 슬라이싱을 사용하여서 1번째부터 11번 전까지 중 10개, 0번은 자신이므로 포함하지 않음
## 문제가 있습니다.. 첫번째 0.3015인데.. 이영화의 인덱스 번호는 0번이다.. 그런데 0번 영화는 아니죠..!!
##ta = aa_sort[1:11]
##ta[0]

## 이 방법은 폐기 이유는 유사도를 정렬은 성공했지만 영화의 정보가 누락이 됨.

np.float64(0.3015117659166547)

In [30]:
## 해결을 위한 방법으로는 정렬을 할 때 정렬전의 인덱스 번호를 포함하여 정렬하면 됩니다..
aa = list(enumerate(cosine_sim[3])) ## enumerate 인덱스를 포함하여 튜플을 만들고
                                    ## list 함수를 이용하여 다시 리스트를 만듬
aa


[(0, np.float64(0.024995115837672686)),
 (1, np.float64(0.0)),
 (2, np.float64(0.0)),
 (3, np.float64(0.9999999999999994)),
 (4, np.float64(0.010433403719159351)),
 (5, np.float64(0.005144601815810792)),
 (6, np.float64(0.012600632435462458)),
 (7, np.float64(0.02695427057891266)),
 (8, np.float64(0.0206522168853895)),
 (9, np.float64(0.13374009066555226)),
 (10, np.float64(0.0)),
 (11, np.float64(0.0)),
 (12, np.float64(0.0)),
 (13, np.float64(0.0)),
 (14, np.float64(0.0)),
 (15, np.float64(0.0040713339225121065)),
 (16, np.float64(0.021121093874993176)),
 (17, np.float64(0.0)),
 (18, np.float64(0.006768893195007469)),
 (19, np.float64(0.010765175685064705)),
 (20, np.float64(0.007178266390761149)),
 (21, np.float64(0.033380775071488206)),
 (22, np.float64(0.0)),
 (23, np.float64(0.0)),
 (24, np.float64(0.019238168304196286)),
 (25, np.float64(0.01701338816136818)),
 (26, np.float64(0.018845673291717255)),
 (27, np.float64(0.0)),
 (28, np.float64(0.008351231142809444)),
 (29, np.float

In [31]:
a = ("apple","banana")
a[1]

'banana'

In [35]:
data_sort = sorted(aa,key=lambda x : x[1],reverse=True)
            ## aa리스트로 정렬해아.. 기준은 각 튜플의 1번째 값으로 정렬해라. 내림차순이다.
f_data = data_sort[1:11]
f_data

[(65, np.float64(0.3015117659166547)),
 (299, np.float64(0.2985704525539681)),
 (428, np.float64(0.2878505467001693)),
 (1359, np.float64(0.26446092382799496)),
 (3854, np.float64(0.1854500300656145)),
 (119, np.float64(0.167996261998507)),
 (2507, np.float64(0.1668289104335827)),
 (9, np.float64(0.13374009066555226)),
 (1181, np.float64(0.1321970213847681)),
 (210, np.float64(0.13045537014449815))]

In [39]:
## f_data
# 6. 5번에서 선정된 영화를 영화의 이름이 아닌 번호입니다. 그래서 이름과 번호를 매칭시키도록 한다.
## f_data의 0번째 번호와 mo_dat의 인덱스명으로 사용하여 출력 
## cosine_sim의 3번째 영화와 유사한 영화의 제목을 출력합니다.
for i in f_data:
    print(i[0],i[1],mo_data[i[0]])
    
    
r_movie = []
for i in f_data:
    b = {'name':mo_data[i[0]]}
    r_movie.append(b)
r_movie    

65 0.3015117659166547 The Dark Knight
299 0.2985704525539681 Batman Forever
428 0.2878505467001693 Batman Returns
1359 0.26446092382799496 Batman
3854 0.1854500300656145 Batman: The Dark Knight Returns, Part 2
119 0.167996261998507 Batman Begins
2507 0.1668289104335827 Slow Burn
9 0.13374009066555226 Batman v Superman: Dawn of Justice
1181 0.1321970213847681 JFK
210 0.13045537014449815 Batman & Robin


[{'name': 'The Dark Knight'},
 {'name': 'Batman Forever'},
 {'name': 'Batman Returns'},
 {'name': 'Batman'},
 {'name': 'Batman: The Dark Knight Returns, Part 2'},
 {'name': 'Batman Begins'},
 {'name': 'Slow Burn'},
 {'name': 'Batman v Superman: Dawn of Justice'},
 {'name': 'JFK'},
 {'name': 'Batman & Robin'}]

In [ ]:
## 위 코드는 코사인 유사도 결과의 3번 행의 정보로 샘플 코드를 만든 것이다
## 즉 영화의 정보가 3이라는 것이다..
## 실제의 서비스는 영화의 번화가 아니라 영화의 제목으로 유사한 영화를 찾는다.
## 즉 The Dark Knight Rises 영화와 유사한 영화를 선정한 것이다 
## 실제 서비스에는 영화의 제목, 즉 The Dark Knight Rises 제목으로 유사한 영화를 찾을 것이다 
## 결과적으로 The Dark Knight Rises 제목으로 해당되는 숫자 인덱스를 알아 내야 합니다

In [23]:
cosine_sim.shape  #행과 열의 사이즈를 알 수 있다.

(4803, 4803)

In [40]:
mo_data ## 인덱스로 값을 찾을 수 있는데 예는 인덱스가 숫자 값이 영화명이죠. 이걸 거꾸로 하면 된다 

0                                         Avatar
1       Pirates of the Caribbean: At World's End
2                                        Spectre
3                          The Dark Knight Rises
4                                    John Carter
                          ...                   
4798                                 El Mariachi
4799                                   Newlyweds
4800                   Signed, Sealed, Delivered
4801                            Shanghai Calling
4802                           My Date with Drew
Name: original_title, Length: 4803, dtype: object

In [42]:
mo_data.index

RangeIndex(start=0, stop=4803, step=1)

In [47]:
find_num = pd.Series(mo_data.index, index=df2['original_title'])
##                      값을 지정     인덱스지정
## 시리즈는 인덱스명과 값으로 이루어진 1차원 데이터... 데이터프레임과 마찬가지로 인덱스명과 번호가 있습니다.
find_num

original_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [ ]:
f ='Batman Forever' ## 입력할 영화 제목 , 클라이언트가 보내준 영화 제목..
number_idx = find_num[f] ## 문자열로 영화의 인덱스명(번호)를 찾는다
aa = list(enumerate(cosine_sim[number_idx])) #3 코사인 유사도 결과에서 인덱스의 행을 가져오는 것이다
data_sort = sorted(aa,key=lambda x : x[1],reverse=True)
            ## aa리스트로 정렬해아.. 기준은 각 튜플의 1번째 값으로 정렬해라. 내림차순이다.
f_data = data_sort[1:11]
#f_data
r_movie = []
for i in f_data:
    b = {'name':mo_data[i[0]]}
    r_movie.append(b)
r_movie  ## 클라이언트에게 응답할 자료 

[{'name': 'The Dark Knight Rises'},
 {'name': 'Batman Begins'},
 {'name': 'The Dark Knight'},
 {'name': 'Batman Returns'},
 {'name': 'Batman & Robin'},
 {'name': 'Batman: The Dark Knight Returns, Part 2'},
 {'name': 'Batman'},
 {'name': 'Cry_Wolf'},
 {'name': 'The Incredible Hulk'},
 {'name': 'Batman v Superman: Dawn of Justice'}]

In [ ]:
### 위는 개발 소스 

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
f = 'Batman Forever'

df1 = pd.read_csv('./data/tmdb_5000_movies.csv')
df2=df1[['original_title','overview']]
co_data = df2['overview']
co_data  ## 유사도를 분석하기 위한 데이터
mo_data = df2['original_title']
mo_data  ## 영화 인덱스와 매칭 되는 영화의 제목을 저장한 데이터
co_data.isnull().values.any() #결과가 True 라면 null값이 있다는 이야기 입니다
# 널값이 존재한다.. 널 값이 있는 행을 지우면 안된다.. 그래서 우리는 빈칸으로 채우기로 함 
co_data = co_data.fillna('')


tf = TfidfVectorizer(stop_words='english') # 백터를 할 때 단어의 기준은 english이다.
tf_matrix = tf.fit_transform(co_data) # overview로 백터화 만들기 

cosine_sim = linear_kernel(tf_matrix, tf_matrix)

find_num = pd.Series(mo_data.index, index=df2['original_title'])
number_idx = find_num[f] ## 문자열로 영화의 인덱스명(번호)를 찾는다
aa = list(enumerate(cosine_sim[number_idx])) #3 코사인 유사도 결과에서 인덱스의 행을 가져오는 것이다
data_sort = sorted(aa,key=lambda x : x[1],reverse=True)
            ## aa리스트로 정렬해아.. 기준은 각 튜플의 1번째 값으로 정렬해라. 내림차순이다.
f_data = data_sort[1:11]
#f_data
r_movie = []
for i in f_data:
    b = {'name':mo_data[i[0]]}
    r_movie.append(b)
r_movie  ## 클라이언트에게 응답할 자료 